In [127]:

#### RECORDING #####
from pynput import mouse
from pynput import keyboard
import time
import pandas as pd
import pyautogui as s

newlist = []

# give a few seconds for user to setup 
s.alert(text='', title='', button='Start Recording')


   

def on_click(x, y, button, pressed):
    if pressed:
        if 'left' in str(button):
            button = 'left'
        else:
            button = 'right'
        newlist.append(['Click',x,y,button,time.time()])
    elif not pressed and newlist != []:
        # if last x,y when pressing different to when release, meaning dragging
        if newlist[-1][1] != x or newlist[-1][2] != y:
            
            newlist[-1][0] = 'MoveTo'
            if 'left' in str(button):
                button = 'left'
            else:
                button = 'right'
            newlist.append(['DragTo',x,y,button,time.time()])
        
        
def on_release(key):
         
    if 'Key' in str(key):
         mykey = str(key)[4:len(str(key))]
         newlist.append(['Key',mykey,'None','None',time.time()])    
    else :
        letter = str(key)[1:2]
        newlist.append(['Write',letter,'None','None',time.time()])
        
    # if users press esc, stop mouse recording as well as keyboard    
    if key == keyboard.Key.esc:
        m_listener.stop()  # stop mouse
        return False       # exit keyboard 

# Collect events until released
with keyboard.Listener(on_release=on_release) as k_listener, \
        mouse.Listener(on_click=on_click) as m_listener:
    k_listener.join()
    m_listener.join()


# create pandas dataframe and to csv
df = pd.DataFrame(newlist, columns = ['Event','X', 'Y','Button','Time'])
df['TimeDiff'] = df['Time'].diff(1)
df = df.fillna(0) # first row of diff will na, so just put zero
df.to_csv(r"C:\Users\r.christianto\MyPython\Screen Record\record.csv",index=False)

In [6]:

#### REPLAY ####


import pyautogui as s
import pandas as pd
import time

s.alert(text='', title='', button='Press OK to Replay')

df = pd.read_csv(r"C:\Users\r.christianto\MyPython\Screen Record\record.csv")
df = df[:-1]
#df['X'] = df['X'].astype(int)
#df['Y'] = df['Y'].astype(int)

list = df.values.tolist()

for i in range(len(list)):
    if 'Click' in list[i][0]:
        x = int(list[i][1])
        y = int(list[i][2])
        button = list[i][3]
        duration = list[i][5]
        s.moveTo(x,y,duration)
        # pause to let web refresh etc
        #time.sleep(duration)
        s.click(button=button)
        
        #time.sleep(0.9)
    elif 'Write' in list[i][0]:
        s.write(list[i][1],0.1)
    elif 'Key' in list[i][0]:
        
        # some of the names need to be changed for pyautogui
        if list[i][1] == 'page_down':
            list[i][1] = 'pgdn'
        elif list[i][1] == 'page_up':
            list[i][1] = 'pgup'
        s.press(list[i][1])
        time.sleep(0.5)
    elif 'MoveTo' in list[i][0]:
        x = int(list[i][1])
        y = int(list[i][2])
        duration = list[i][5]
        print (x,y,duration)
        s.moveTo(x,y,duration)
        # pause to let web refresh etc
        #time.sleep(duration)
       
    elif 'DragTo' in list[i][0]:
        x = int(list[i][1])
        y = int(list[i][2])
       
        s.dragTo(x,y,1)

s.alert("Replay completed")

3024 123 2.1803884506225586


'OK'